## this is the companion notebbok to the [mimikit : Step-by-step tutorial]()

it shows how to use `mimikit` with **Gdrive** and can be executed in [colab](https://colab.research.google.com/github/k-tonal/mimikit-notebooks/blob/main/Step-by-step%20Gdrive.ipynb)

## Install `mimikit`

In [ ]:
!pip install mimikit

## Connect to your Gdrive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My\ Drive

## Import classes & functions from `mimikit`

In [1]:
from mimikit import get_trainer
from mimikit.data import Database
from mimikit.freqnet import freqnet_db, FreqNet

## If you don't have audio files on Gdrive, upload some here

In [ ]:
from google.colab import files

uploaded = files.upload()

## Define some variables for configuration

In [ ]:
path_to_your_files = "./"
db_filename = "demo_db.h5"
path_to_model = 'demo_model/'

## Make a database

In [ ]:
db = freqnet_db(db_filename, roots=[path_to_your_files])

## Define a model and some training options

In [ ]:
model = FreqNet(data_object=db.fft,
                model_dim=1024,
                groups=1,
                n_layers=(4,),
                with_skip_conv=False,
                with_residual_conv=False,
                accum_outputs=0,
                concat_outputs=0,
                pad_input=0)

trainer = get_trainer(max_epochs=100,
                      epochs=[30, 60, 99],
                      root_dir=path_to_model)

## Launch the training (and go do something else!...)

In [ ]:
trainer.fit(model)

## Generate audios with the model

In [ ]:
from mimikit.utils import audio

prompt = model.random_train_example()
output = model.generate(prompt, n_steps=2048)

audio(output.squeeze().numpy())

## Save the generated outputs to your Gdrive

In [ ]:
name_for_output = "wow_cool_output"

model.log_audio(name_for_output, output)

## Reload a checkpoint

In [ ]:
epoch = 99

path_to_ckpt = path_to_model + "states/epoch=%i.ckpt" % epoch

model = FreqNet(path_to_ckpt, data_object=db.fft)

model.hparams